In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [5]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    all_stats_2021 = stats("https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats", 2021)
    all_stats_2020 = stats("https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats", 2020)
    all_stats_2019 = stats("https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats", 2019)
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    overall_stats.append(all_stats_2021)
    overall_stats.append(all_stats_2020)
    overall_stats.append(all_stats_2019)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [6]:
overall_stats_df = concate()

In [7]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Liverpool,22,15,6,1,51,19,32,51,2.32,48.6,24.6,24.0,1.09,47835,Mohamed Salah - 14,Alisson,NaN,59.7,48,2.18,38,1.73,36.4,6.50,0.31,34.5,2024
1,2,Manchester City,21,14,4,3,51,24,27,46,2.19,41.5,20.6,20.9,1.00,47967,Erling Haaland - 14,Ederson,NaN,64.5,50,2.38,38,1.81,23.8,6.38,0.34,38.6,2024
2,3,Arsenal,22,14,4,4,44,21,23,46,2.09,40.8,17.1,23.7,1.08,60180,Bukayo Saka - 7,David Raya,NaN,60.9,43,1.95,30,1.36,36.4,4.91,0.34,30.1,2024
3,4,Tottenham,22,13,4,5,47,33,14,43,1.95,39.3,37.1,2.2,0.10,61586,Son Heung-min - 12,Guglielmo Vicario,NaN,59.7,44,2.00,37,1.68,22.7,5.77,0.34,35.9,2024
4,5,Aston Villa,22,13,4,5,44,30,14,43,1.95,38.6,29.3,9.3,0.42,41558,Ollie Watkins - 10,Emiliano Martínez,NaN,55.5,41,1.86,31,1.41,22.7,5.05,0.33,35.0,2024


In [8]:
overall_stats_df.shape

(120, 29)

In [9]:
overall_stats_df.to_csv("../datasets/overall_stats2.csv", index=False)